<a href="https://colab.research.google.com/github/ehdghks316/itwill_studyGroup_D/blob/main/4_11%ED%8C%8C%EC%9D%B4%EC%8D%AC(CountVectorizer%2C_TF_IDF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
★ word embedding
- 일상의 언어를 컴퓨터가 이해할 수 있는 벡터(숫자)로 표현하는 방법

★ BOW
- 문서를 숫자 벡터로 변환하는 가장 기본적인 방법은 BOW(Bag Of Words) 인코딩 방법

★ 문서 단어 행렬(Document Term Matrix)
- 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현
(행이 문서들 열이 단어들)

with open('c:/data/mom.txt','r') as file:
    mom = file.read()


[문제193] mom.txt 데이터를 가지고 화면과 같은 문서 단어행렬을 수동으로 생성해주세요.
    am and be .... take to ...
 0   0   1   0 ...  0    1 ...
 1   ...
 2   ...

# \n 단어 공백으로 변경 
mom = mom.replace('\n',' ')
# ' '으로 단어 나누기
mom_split = mom.split(' ')
# dictionary 형태로 중복없이 추출
mom_dict = {}

for i in mom_split:
    if i == '':
        continue
    elif i not in mom_dict.keys():
        mom_dict[i] = 1
    else:
        continue

mom_dict

# dictionary 형태로 중복없이 추출2
mom_split.remove('') # 먼저 공백만 있는 리스트를 추출
for i in mom_split:
    if i not in mom_dict.keys():
        mom_dict[i] = 1
    else:
        continue
    
'''
# 단어들을 컬럼으로 2
import pandas as pd
from pandas import DataFrame, Series

df = DataFrame(mom_dict.keys()).T
df.loc[1] = mom_dict.values() # df.loc 한 행 추가
df = df.drop(0)
df = df.reset_index()
'''

# 단어들을 컬럼으로 
mom_dtm = DataFrame(mom_dict.values(),mom_dict.keys()).T


# 강사님 풀이
mom = pd.read_csv('c:/data/mom.txt',header=None,names=['sentens'])

mom['sentens'] = mom['sentens'].str.lower()
mom['sentens'].str.count('mommy').sum() # 각 문서별로 mommy가 들어있는 숫자.sum() -> 다 합쳐서 몇개있는지 확인
mom['sentens'].str.count('i').sum() # count메소드는 단어별로 세는것이 아니라 i 철자를 모두 추출
mom['sentens'].str.findall('i')

data = ' '.join(mom['sentens'])
words = set(data.split())
words = [i for i in words if len(i) >= 2]
words.sort() # sort() 오름차순 정렬 바로적용
words

freq_table = {}
for i in words:
    freq_table.setdefault(i,0)

for j in mom['sentens'][1].split():
    if j in freq_table.keys():
        freq_table[j] += 1
        
freq_df = pd.DataFrame()

temp = DataFrame(Series(freq_table)).T
freq_df = pd.concat([freq_df, temp] ,ignore_index=True)

temp = pd.DataFrame.from_dict([freq_table])
freq_df = pd.concat([freq_df, temp] ,ignore_index=True)

### 강사님 풀이 최종
mom = pd.read_csv('c:/data/mom.txt',header=None,names=['sentens'])
mom['sentens'] = mom['sentens'].str.lower()

data = ' '.join(mom['sentens'])
words = set(data.split())
words = [i for i in words if len(i) >= 2]
words.sort() # sort() 오름차순 정렬 바로적용
words


freq_df = pd.DataFrame()
for i in mom['sentens']:
    freq_table = {}
    for w in words:
        freq_table.setdefault(w,0)
        
    for j in i.split():
        if j in freq_table.keys():
            freq_table[j] += 1
    temp = pd.DataFrame.from_dict([freq_table])    
    freq_df = pd.concat([freq_df,temp],ignore_index=True)

freq_df.sum(axis=0) # 컬럼별로 합
freq_df.sum(axis=1) # 행(문서)별로 합

freq_df['document'] = mom['sentens']
del freq_df['document']

# 특정한 인덱스 열 위치에 새로운 열을 추가할 때
freq_df.insert(0,'document',mom['sentens'])
freq_df.iloc[0]

freq_df[words].sum()
freq_df[words].sum().sort_values(ascending=False)

freq_df.columns.difference(['document']) # document열만 제외하고 나머지 열만 보겠어 ( 특정한 열 제외)
freq_df[freq_df.columns.difference(['document'])].sum().sum()

########################################################################################

★ CountVectorizer
- 문서집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW인코딩 벡터를 만든다.

import operator
from sklearn.feature_extraction.text import CountVectorizer
mom = pd.read_csv('c:/data/mom.txt',header=None,names=['sentens'])


vect = CountVectorizer() #한글자 자동제거, 자동 소문자 변환
vect.fit(mom['sentens']) # words를 만드는 작업(단어 사전)
vect.vocabulary_ # 각단어 뒤에 value값은 인덱스(빈도수x)
sorted(vect.vocabulary_.items(),key=operator.itemgetter(1))
vect.get_feature_names() # 단어들만 추출

feature_vector = vect.transform(mom['sentens'])
feature_vector
feature_vector.shape # 몇 행 몇 열인지 확인

df = pd.DataFrame(feature_vector.toarray(), columns=vect.get_feature_names()) # toarray() : array형태로 확인 -> dataframe으로 바로 생성가능
df.sum(axis=0).sort_values(ascending=False)


vect = CountVectorizer()
feature_vector = vect.fit_transform(mom['sentens']) # fit, transform 같이 사용
feature_vector
vect.vocabulary_
vect.get_feature_names()
df = pd.DataFrame(feature_vector.toarray(), columns=vect.get_feature_names()) # toarray() : array형태로 확인 -> dataframe으로 바로 생성가능
df.sum(axis=0).sort_values(ascending=False)


★ 불용어
- 의미없는 단어를 제외

stopwords = ['am','are','be','and','is','the','then']
vect = CountVectorizer(stop_words=stopwords)
feature_vector = vect.fit_transform(mom['sentens']) # fit, transform 같이 사용
feature_vector
vect.vocabulary_
vect.get_feature_names()
df = pd.DataFrame(feature_vector.toarray(), columns=vect.get_feature_names()) # toarray() : array형태로 확인 -> dataframe으로 바로 생성가능
df.sum(axis=0).sort_values(ascending=False)


vect = CountVectorizer(stop_words='english') # english 불용어 사전
feature_vector = vect.fit_transform(mom['sentens']) # fit, transform 같이 사용
feature_vector
vect.vocabulary_
vect.get_feature_names()
df = pd.DataFrame(feature_vector.toarray(), columns=vect.get_feature_names()) # toarray() : array형태로 확인 -> dataframe으로 바로 생성가능
df.sum(axis=0).sort_values(ascending=False)

★ 토큰

vect = CountVectorizer(analyzer='word') # 공백문자를 기준으로 분리(기본값)
feature_vector = vect.fit_transform(mom['sentens']) # fit, transform 같이 사용
feature_vector
vect.vocabulary_
vect.get_feature_names()
df = pd.DataFrame(feature_vector.toarray(), columns=vect.get_feature_names()) # toarray() : array형태로 확인 -> dataframe으로 바로 생성가능
df.sum(axis=0).sort_values(ascending=False)

vect = CountVectorizer(analyzer='char') # anlayzer='char'는 알파벳 철자를 기준으로 나눔
feature_vector = vect.fit_transform(mom['sentens']) # fit, transform 같이 사용
feature_vector
vect.vocabulary_
vect.get_feature_names()
df = pd.DataFrame(feature_vector.toarray(), columns=vect.get_feature_names()) # toarray() : array형태로 확인 -> dataframe으로 바로 생성가능
df.sum(axis=0).sort_values(ascending=False)

vect = CountVectorizer(token_pattern='t\w+') # token_pattern='정규표현식' , 여기서는 t로시작하는 단어들 추출, 정규표현식을 활용해서 내가 원하는 글자 패턴의 사전을 만들 수 있다.
feature_vector = vect.fit_transform(mom['sentens']) # fit, transform 같이 사용
feature_vector
vect.vocabulary_
vect.get_feature_names()
df = pd.DataFrame(feature_vector.toarray(), columns=vect.get_feature_names()) # toarray() : array형태로 확인 -> dataframe으로 바로 생성가능
df.sum(axis=0).sort_values(ascending=False)

★ n-gram(n그램)
- 1개의 단어로 하나의 토큰을 생성 1-gram, monogram
- 2개의 단어로 하나의 토큰을 생성 2-gram, bigram
- 3개의 단어로 하나의 토큰을 생성 3-gram, trigram

#vect = CountVectorizer(ngram_range=(1,1)) # 1-gram,monogram 기본값
#vect = CountVectorizer(ngram_range=(2,2)) # 2-gram, bigram
vect = CountVectorizer(ngram_range=(1,2)) # 1-grma, 2-gram 둘 다 생성
feature_vector = vect.fit_transform(mom['sentens']) # fit, transform 같이 사용
feature_vector
vect.vocabulary_
vect.get_feature_names()
df = pd.DataFrame(feature_vector.toarray(), columns=vect.get_feature_names()) # toarray() : array형태로 확인 -> dataframe으로 바로 생성가능
df.sum(axis=0).sort_values(ascending=False)

vect = CountVectorizer(ngram_range=(1,2), token_pattern='t\w+') 
feature_vector = vect.fit_transform(mom['sentens']) # fit, transform 같이 사용
feature_vector
vect.vocabulary_
vect.get_feature_names()m
df = pd.DataFrame(feature_vector.toarray(), columns=vect.get_feature_names()) # toarray() : array형태로 확인 -> dataframe으로 바로 생성가능
df.sum(axis=0).sort_values(ascending=False)


★ 빈도수
토큰의 빈도가 max_df로 지정한 값을 초과하거나 min_df로 지정한 값보다 작은 경우에는 무시한다.

vect = CountVectorizer(analyzer='word', max_df=3, min_df=2) # max_df=초과, min_df=이하 
feature_vector = vect.fit_transform(mom['sentens']) # fit, transform 같이 사용
feature_vector
vect.vocabulary_
vect.get_feature_names()
df = pd.DataFrame(feature_vector.toarray(), columns=vect.get_feature_names()) # toarray() : array형태로 확인 -> dataframe으로 바로 생성가능
df.sum(axis=0).sort_values(ascending=False)


★ TF-IDF(Term Frequency - Inverse Document Frequency)
# TF(Term Frequency)
-한 개 문서안에서 특정 단어의 등장 빈도수

vect = CountVectorizer(analyzer='word') # max_df, min_df 
feature_vector = vect.fit_transform(mom['sentens']) # fit, transform 같이 사용
feature_vector
vect.vocabulary_
vect.get_feature_names()
df = pd.DataFrame(feature_vector.toarray(), columns=vect.get_feature_names()) # toarray() : array형태로 확인 -> dataframe으로 바로 생성가능
df.sum(axis=0).sort_values(ascending=False)

# DF(Document Frequency)
- 특정 단어가 나타나는 문서의 수
- 문서빈도
- DF가 클수록 여러 문서에 흔하게 사용된 일반적인 단어라고 할 수 있다.

0      Mommy  I love you And I want to say thank you
1   I am happy  everyday for the love you give to me
2              Sometimes I am sad Sometimes I do cry
3   Then I just remember  Mommy how much you love me
4  Mommy you take care of me you help me to be st...

mommy
TF = ? [문서1] 1, [문서2] 0, [문서3] 1, [문서4] 1 , [문서5] 1
DF = ? 4

you
TF = ? [문서1] 2, [문서2] 1, [문서3] 0, [문서4] 1, [문서5] 2
DF = ? 4

# IDF(Inverse Document Frequency)
- DF에 역수로 변환을 해준 값을 의미한다. 
- DF의 역수이므로 DF가 클수록 IDF값은 작아지고 DF작을수록 IDF 커진다.
- 여러 문서에서 자주 등장하는 단어들에 대해서 패널티를 주는 방법(a, the 같이 의미없이 자주 등장하는 단어)
- IDF-SMOOTHING에 따라 달라진다.

TF-IDF = TF * IDF

IDF = log(N/DF), log(N/DF)+1, log((N+1)/(DF+1)) # 세가지 알고리즘이 사용된다.
N : 전체 문서의 수
DF : 특정 단어가 나타나는 문서의 수

# 내 문서에는 두드러지게 나오지만 다른 문서에는 나오지 않는 것이 의미가 있다. / a, the 같은 자주나오지만 의미없는 단어들이 중요한 것처럼 많이 나오는 것이 문제(countvectorizer에서)
# countvectorizer만 보고 하면 안되고 TF-IDF까지 봐야 의미있는 결과를 도출할 수 있다.(위에 줄 같은 문제가 생기기 때문에)
# idf는 패널티 값으로 패널티를 준다고 생각
# tf * idf 계산시 파급효과가 높아질 수도 있고 낮아질 수도 있다.


from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tf_idf_vect = tfidf_vect.fit(mom['sentens'])
tf_idf_vect.vocabulary_ # term을 추출하는 함수
tf_idf_vect.get_feature_names() 
tf_idf_vect.idf_ # 각 feature의 idf 값

tfidf_vect.transform(mom['sentens']).toarray()

pd.DataFrame(tf_idf_vect.idf_,index=tf_idf_vect.get_feature_names(),columns=['idf'])

poem = pd.read_csv('c:/data/poem.txt',header=None, names=['sentens'])
poem

vect = CountVectorizer()
vect.fit(poem['sentens'])
vect.vocabulary_
vect.get_feature_names()

feature_vector = vect.transform(poem['sentens'])
feature_vector
df = pd.DataFrame(feature_vector.toarray(),columns=vect.get_feature_names())
df
df.sum(axis=0)
# 조사들, 의미있는 한 글자들이 나오지 않는 문제가 있음
# 형태소 분석이 필요함

from konlpy.tag import Okt
okt = Okt()

def okt_pos(arg):
    token_corpus = []
    for i in okt.pos(arg):
        if i[1] in ['Noun','Adjective']:
            token_corpus.append(i[0])
    return token_corpus

[okt_pos(i) for i in poem['sentens']]

cv = CountVectorizer(tokenizer=okt_pos)
cv_trans = cv.fit_transform(poem['sentens'])
cv.vocabulary_
cv.get_feature_names()
cv_trans.toarray()
df = pd.DataFrame(cv_trans.toarray(),columns=cv.get_feature_names())
df
# 한글일 경우에는 형태소 분석을 해서 해야한다. 위처럼
# n-gram도 생각해보기

# 이 문서에만 두드러지게 나타나는 것을 찾겠다. tf-idf 이용해서
# countvectorizer를 tf-idf로 변환하는 작업
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_trans = TfidfTransformer()
tf_idf_vect = tfidf_trans.fit_transform(cv_trans)
tf_idf_vect.toarray()

df_tfidf = pd.DataFrame(tf_idf_vect.toarray(),columns=cv.get_feature_names())
df_tfidf


[문제194] 중앙일보 인공지능 뉴스 기사 검색 데이터를 이용해서 tf-idf를 생성해주세요.
각각의 기사별로 빈도수를 확인. 다른 기사에 없는 단어들을 찾기.

news = pd.read_csv('c:/data/joongang_ai_news_df.csv')
news
news.news
'''
tfidf_vect = TfidfVectorizer()
tfidf_vect_t = tfidf_vect.fit(news.news)
tfidf_vect_t.vocabulary_
tfidf_vect_t.get_feature_names()
tfidf_vect_t.idf_

tfidf_vect_trans = tfidf_vect_t.transform(news.news).toarray()
tfidf_vect_trans
pd.DataFrame(tfidf_vect_trans,columns=tfidf_vect_t.get_feature_names())
'''
# countvectorizer
from konlpy.tag import Okt
okt = Okt()
def okt_pos(arg):
    token_corpus = []
    for i in okt.pos(arg):
        if i[1] in ['Noun','Adjective']:
            token_corpus.append(i[0])
    return token_corpus

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(tokenizer=okt_pos)
cv_trans = cv.fit_transform(news.news)
cv.vocabulary_
cv.get_feature_names()
cv_trans.toarray()
df = pd.DataFrame(cv_trans.toarray(),columns=cv.get_feature_names())

# tfidf 변환
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_trans = TfidfTransformer()
tf_idf_vect = tfidf_trans.fit_transform(cv_trans) # cv_trans는 countvectorizer를 만들어 놓은 변수
tf_idf_vect.toarray()
df_tfidf = pd.DataFrame(tf_idf_vect.toarray(),columns=cv.get_feature_names())
df_tfidf

# 10개 기사만 추출해서 수행
data = news.news[0:10]
cv = CountVectorizer(tokenizer=okt_pos, stop_words=['것','수','등'])
cv_t = cv.fit_transform(data)
cv.vocabulary_
cv.get_feature_names()
cv_t.toarray()
x = pd.DataFrame(cv_t.toarray(),columns=cv.get_feature_names())
x.sum(axis=0).sort_values(ascending=False)

tf_idf_trans = TfidfTransformer()
tf_idf_trans_vect = tf_idf_trans.fit_transform(cv_t)
tf_idf_trans_vect.toarray()
df_tf_idf_trans_vect = pd.DataFrame(tf_idf_trans_vect.toarray(),columns=cv.get_feature_names())

df_tf_idf_trans_vect.iloc[0].idxmax()
for i in range(len(df_tf_idf_trans_vect)):
    print(i,df_tf_idf_trans_vect.iloc[i].idxmax())


# 강사님
from konlpy.tag import Okt
okt = Okt()
def okt_pos(arg):
    token_corpus = []
    for i in okt.pos(arg):
        if i[1] in ['Noun','Adjective']:
            token_corpus.append(i[0])
    token_corpus = [x fo x in token_corpus if len(x) >1]
    return token_corpus

cv = CountVectorizer(tokenizer=okt_pos)
cv_t = cv.fit_transform(data)
df = pd.DataFrame(cv_t.toarray(),columns=cv.get_feature_names())
df.iloc[0].max()
df.iloc[0].idxmax()
df.loc[0,'연구']
df.iloc[0].sort_values(ascending=False)[:10]

df.iloc[1].max()
df.iloc[1].idxmax() # 가장 큰 값의 인덱스구하는 함수
df.loc[1,'연구']
df.iloc[1].sort_values(ascending=False)[:10]

for i in range(len(df)):
    print(i, df.iloc[i].idxmax())
    
import matplotlib.pylab as plt
from matplotlib import font_manager,rc
font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)
from wordcloud import WordCloud
w = WordCloud(font_path = 'c:/Windows/Fonts/malgun.ttf',
              width=1000,height=1000,background_color='white').generate_from_frequencies(dict(df.sum(axis=0)))
plt.imshow(w)
plt.axis('off')
w.to_file('c:/data/contents_noun_freq.jpg')

import numpy as np

